In [1]:
from marubatsu import Marubatsu
from ai import ai_by_mmscore, dprint
from copy import deepcopy

@ai_by_mmscore
def ai_pvs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None,
               eval_params={}, tt=None, tt_for_mo=None):
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        boardtxt = mborig.board_to_str()
        if boardtxt in tt:
            lower_bound, upper_bound, _ = tt[boardtxt]
            if lower_bound == upper_bound:
                return lower_bound
            elif upper_bound <= alpha:
                return upper_bound
            elif beta <= lower_bound:
                return lower_bound
            else:
                alpha = max(alpha, lower_bound)
                beta = min(beta, upper_bound)
        else:
            lower_bound = min_score
            upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                if abscore > score:
                    bestmove = (x, y)
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                if abscore < score:
                    bestmove = (x, y)
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        boardtxtlist = calc_same_boardtexts(mborig, bestmove)
        if score <= alphaorig:
            upper_bound = score
        elif score < betaorig:
            lower_bound = score
            upper_bound = score
        else:
            lower_bound = score
        for boardtxt, move in boardtxtlist:
            tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [2]:
@ai_by_mmscore
def ai_pvs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None, eval_params={},
           tt=None, tt_for_mo=None):
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        boardtxt = mborig.board_to_str()
        if boardtxt in tt:
            lower_bound, upper_bound, _ = tt[boardtxt]
            if lower_bound == upper_bound:
                return lower_bound
            elif upper_bound <= alpha:
                return upper_bound
            elif beta <= lower_bound:
                return lower_bound
            else:
                alpha = max(alpha, lower_bound)
                beta = min(beta, upper_bound)
        else:
            lower_bound = min_score
            upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]
        if mborig.turn == Marubatsu.CIRCLE:
            x, y = legal_moves[0]
            mb = deepcopy(mborig)
            mb.move(x, y)
            score = ab_search(mb, depth + 1, tt, alpha, beta)
            alpha = max(alpha, score)
            bestmove = (x, y)
            if score < beta:
                for x, y in legal_moves[1:]:
                    mb = deepcopy(mborig)
                    mb.move(x, y)
                    abscore = ab_search(mb, depth + 1, tt, alpha, alpha + 1)
                    if abscore > score:
                        bestmove = (x, y)
                    score = max(score, abscore)
                    if score >= beta:
                        break
                    elif score > alpha:
                        abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                        if abscore > score:
                            bestmove = (x, y)
                        score = max(score, abscore)
                        if score >= beta:
                            break
                        alpha = max(alpha, score)                    
        else:
            x, y = legal_moves[0]
            mb = deepcopy(mborig)
            mb.move(x, y)
            score = ab_search(mb, depth + 1, tt, alpha, beta)
            beta = min(beta, score)
            bestmove = (x, y)
            if score > alpha:
                for x, y in legal_moves[1:]:
                    mb = deepcopy(mborig)
                    mb.move(x, y)
                    abscore = ab_search(mb, depth + 1, tt, beta - 1, beta)
                    if abscore < score:
                        bestmove = (x, y)
                    score = min(score, abscore)
                    if score <= alpha:
                        break
                    elif score < beta:
                        abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                        if abscore < score:
                            bestmove = (x, y)
                        score = min(score, abscore)
                        if score <= alpha:
                            break
                        beta = min(beta, score)   
                    
        from util import calc_same_boardtexts

        boardtxtlist = calc_same_boardtexts(mborig, bestmove)
        if score <= alphaorig:
            upper_bound = score
        elif score < betaorig:
            lower_bound = score
            upper_bound = score
        else:
            lower_bound = score
        for boardtxt, move in boardtxtlist:
            tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [3]:
from ai import ai_abs_dls, ai14s

eval_params = {"minimax": True}

mb = Marubatsu()
result_abs = ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params,
                        use_tt=True, analyze=True)
print(result_abs["count"])

tt = result_abs["tt"]
result_abs_mo = ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params, 
                           use_tt=True, tt_for_mo=tt, analyze=True)
print(result_abs_mo["count"])

result_pvs = ai_pvs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params,
                    tt_for_mo=tt, analyze=True)
print(result_pvs["count"])

1175
538
538


In [4]:
mb = Marubatsu()
mb.move(0, 0)
result_abs = ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params,
                        use_tt=True, analyze=True)
print(result_abs["count"])

tt = result_abs["tt"]
result_abs_mo = ai_abs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params, 
                           use_tt=True, tt_for_mo=tt, analyze=True)
print(result_abs_mo["count"])

result_pvs = ai_pvs_dls(mb, maxdepth=9, eval_func=ai14s, eval_params=eval_params,
                    tt_for_mo=tt, analyze=True)
print(result_pvs["count"])

876
573
569


In [5]:
from time import perf_counter

def ai_pvs_iddfs(mb, debug=False, timelimit=10, eval_func=None,
                eval_params={}, analyze=False):   
    starttime = perf_counter()
    timelimit_pc = starttime + timelimit
    tt_for_mo = {}
    bestmove = mb.calc_legal_moves()[0]
    totalcount = 0
    resultlist = []
    for maxdepth in range(9 - mb.move_count):
        try:
            result = ai_pvs_dls(mb, maxdepth=maxdepth, timelimit_pc=timelimit_pc,
                                eval_func=eval_func, eval_params=eval_params,
                                analyze=True, tt_for_mo=tt_for_mo)
        except RuntimeError:
            dprint(debug, "time out")
            break
        tt_for_mo = result["tt"]
        totaltime = perf_counter() - starttime
        resultlist.append(result)
        candidate = result["candidate"]
        bestmove = result["bestmove"]
        count = result["count"]
        time = result["time"]
        totalcount += count
        dprint(debug, f"maxdepth: {maxdepth}, time: {time:6.2f}/{totaltime:6.2f} ms, count {count:5}/{totalcount:5}, bestmove: {bestmove}, candidate: {candidate}")
    dprint(debug, f"totaltime: {perf_counter() - starttime: 6.2f} ms")
    if analyze:
        return resultlist
    else:
        return bestmove

In [6]:
from ai import ai_ab_iddfs

mb = Marubatsu()
ai_ab_iddfs(mb, timelimit=10, debug=True, eval_func=ai14s,
            eval_params=eval_params, analyze=True, moveordering=True)
ai_pvs_iddfs(mb, timelimit=10, debug=True, eval_func=ai14s,
             eval_params=eval_params, analyze=True);

maxdepth: 0, time:   0.00/  0.00 ms, count     9/    9, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 1, time:   0.01/  0.01 ms, count    33/   42, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 2, time:   0.01/  0.03 ms, count    66/  108, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 3, time:   0.02/  0.05 ms, count   160/  268, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 4, time:   0.03/  0.07 ms, count   269/  537, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 5, time:   0.03/  0.10 ms, count   447/  984, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 6, time:   0.09/  0.20 ms, count   578/ 1562, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 7, time:   0.05/  0.25 ms, count   548/ 2110, bestmove: (0, 1), candidate: [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
maxdepth: 8, time:   0.07/  0.31 ms, count   542/ 2652, bestmove: (0, 0), candidate: [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
totaltime:   0.31 ms
maxde

In [7]:
from copy import deepcopy
from functools import wraps
from time import perf_counter
from random import choice

def ai_by_mmscore(eval_func):
    @wraps(eval_func)
    def wrapper(mb_orig, debug=False, *args, rand=True, share_tt=True,
                analyze=False, calc_score=False, **kwargs):       
        if calc_score:
            score, count = eval_func(mb_orig, debug, *args, **kwargs)
            return score
        
        starttime = perf_counter()
        dprint(debug, "Start ai_by_mmscore")
        dprint(debug, mb_orig)
        legal_moves = mb_orig.calc_legal_moves()
        dprint(debug, "legal_moves", legal_moves)
        maxnode = mb_orig.turn == Marubatsu.CIRCLE
        best_score = float("-inf") if maxnode else float("inf")
        best_moves = []
        tt = {} if share_tt else None
        totalcount = 0
        if analyze:
            score_by_move = {}
        for move in legal_moves:
            dprint(debug, "=" * 20)
            dprint(debug, "move", move)
            mb = deepcopy(mb_orig)
            x, y = move
            mb.move(x, y)
            dprint(debug, mb)
            score, count = eval_func(mb, debug, tt=tt, *args, **kwargs)
            totalcount += count
            dprint(debug, "score", score, "best score", best_score)
            if analyze:
                score_by_move[move] = score
            
            if (maxnode and best_score < score) or (not maxnode and best_score > score):
                best_score = score
                best_moves = [move]
                dprint(debug, "UPDATE")
                dprint(debug, "  best score", best_score)
                dprint(debug, "  best moves", best_moves)
            elif best_score == score:
                best_moves.append(move)
                dprint(debug, "APPEND")
                dprint(debug, "  best moves", best_moves)

        dprint(debug, "=" * 20)
        dprint(debug, "Finished")
        dprint(debug, "best score", best_score)
        dprint(debug, "best moves", best_moves)
        bestmove = choice(best_moves) if rand else best_moves[0]
        if analyze:
            if share_tt:
                PV = []
                mb = deepcopy(mb_orig)
                while mb.status == Marubatsu.PLAYING:
                    PV.append(bestmove)
                    x, y = bestmove
                    if mb.board[x][y] != Marubatsu.EMPTY:
                        print("そのマスには着手済みです")
                        break
                    mb.move(x, y)
                    boardtxt = mb.board_to_str()
                    if boardtxt in tt:
                        _, _, bestmove = tt[boardtxt]
                    else:
                        break                
            else:
                PV = bestmove
            return {
                "candidate": best_moves,
                "score_by_move": score_by_move,
                "tt": tt,
                "time": perf_counter() - starttime,
                "bestmove": PV[0],
                "score": best_score,
                "count": totalcount,
                "PV": PV,
            }
        else:
            return bestmove
        
    return wrapper

In [8]:
@ai_by_mmscore
def ai_pvs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None, eval_params={},
           tt=None, tt_for_mo=None):
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        boardtxt = mborig.board_to_str()
        if boardtxt in tt:
            lower_bound, upper_bound, _ = tt[boardtxt]
            if lower_bound == upper_bound:
                return lower_bound
            elif upper_bound <= alpha:
                return upper_bound
            elif beta <= lower_bound:
                return lower_bound
            else:
                alpha = max(alpha, lower_bound)
                beta = min(beta, upper_bound)
        else:
            lower_bound = min_score
            upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]
        if mborig.turn == Marubatsu.CIRCLE:
            x, y = legal_moves[0]
            mb = deepcopy(mborig)
            mb.move(x, y)
            score = ab_search(mb, depth + 1, tt, alpha, beta)
            alpha = max(alpha, score)
            bestmove = (x, y)
            if score < beta:
                for x, y in legal_moves[1:]:
                    mb = deepcopy(mborig)
                    mb.move(x, y)
                    abscore = ab_search(mb, depth + 1, tt, alpha, alpha + 1)
                    if abscore > score:
                        bestmove = (x, y)
                    score = max(score, abscore)
                    if score >= beta:
                        break
                    elif score > alpha:
                        abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                        if abscore > score:
                            bestmove = (x, y)
                        score = max(score, abscore)
                        if score >= beta:
                            break
                        alpha = max(alpha, score)                    
        else:
            x, y = legal_moves[0]
            mb = deepcopy(mborig)
            mb.move(x, y)
            score = ab_search(mb, depth + 1, tt, alpha, beta)
            beta = min(beta, score)
            bestmove = (x, y)
            if score > alpha:
                for x, y in legal_moves[1:]:
                    mb = deepcopy(mborig)
                    mb.move(x, y)
                    abscore = ab_search(mb, depth + 1, tt, beta - 1, beta)
                    if abscore < score:
                        bestmove = (x, y)
                    score = min(score, abscore)
                    if score <= alpha:
                        break
                    elif score < beta:
                        abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                        if abscore < score:
                            bestmove = (x, y)
                        score = min(score, abscore)
                        if score <= alpha:
                            break
                        beta = min(beta, score)   
                    
        from util import calc_same_boardtexts

        boardtxtlist = calc_same_boardtexts(mborig, bestmove)
        if score <= alphaorig:
            upper_bound = score
        elif score < betaorig:
            lower_bound = score
            upper_bound = score
        else:
            lower_bound = score
        for boardtxt, move in boardtxtlist:
            tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [9]:
result = ai_pvs_dls(mb, eval_func=ai14s, eval_params=eval_params, maxdepth=9, analyze=True)
print(result["bestmove"], result["PV"])
for x, y in result["PV"]:
    print(f"({x}, {y})")
    mb.move(x, y)
    print(mb)

(2, 2) [(2, 2), (1, 1), (2, 1), (2, 0), (0, 2), (1, 2), (1, 0), (0, 1), (0, 0)]
(2, 2)
Turn x
...
...
..O

(1, 1)
Turn o
...
.X.
..o

(2, 1)
Turn x
...
.xO
..o

(2, 0)
Turn o
..X
.xo
..o

(0, 2)
Turn x
..x
.xo
O.o

(1, 2)
Turn o
..x
.xo
oXo

(1, 0)
Turn x
.Ox
.xo
oxo

(0, 1)
Turn o
.ox
Xxo
oxo

(0, 0)
winner draw
Oox
xxo
oxo



In [10]:
a = set()
a.add("o....")
a.add(".o...")
a.add("..o..")
a.add("...o.")
a.add("....o")
print(a)

{'....o', 'o....', '..o..', '.o...', '...o.'}


In [11]:
for data in a:
    print(data)

....o
o....
..o..
.o...
...o.


In [12]:
for data in a:
    print(data)

....o
o....
..o..
.o...
...o.


In [1]:
# JupyterLab を再起動後に下記のプログラムを実行する
a = set()
a.add("o....")
a.add(".o...")
a.add("..o..")
a.add("...o.")
a.add("....o")
print(a)

{'...o.', '..o..', '....o', '.o...', 'o....'}


In [2]:
# JupyterLab を再起動せずにに下記のプログラムを実行する
a = set()
a.add("o....")
a.add(".o...")
a.add("..o..")
a.add("...o.")
a.add("....o")
print(a)

{'...o.', '..o..', '....o', '.o...', 'o....'}


In [3]:
from util import calc_same_boardtexts
from marubatsu import Marubatsu

mb = Marubatsu()
mb.move(1, 1)
print(mb)
boardtxtlist = calc_same_boardtexts(mb, (0, 0))
print("boardtxtlist")
print(boardtxtlist)
tt = {}
print("登録後の置換表の値")
for boardtxt, move in boardtxtlist:
    tt[boardtxt] = (0, 0, move)
    print(tt)

Turn x
...
.O.
...

boardtxtlist
{('....o....', (2, 2)), ('....o....', (2, 0)), ('....o....', (0, 0)), ('....o....', (0, 2))}
登録後の置換表の値
{'....o....': (0, 0, (2, 2))}
{'....o....': (0, 0, (2, 0))}
{'....o....': (0, 0, (0, 0))}
{'....o....': (0, 0, (0, 2))}


In [1]:
# JupyterLab を再起動後に下記のプログラムを実行する
from marubatsu import Marubatsu
from util import calc_same_boardtexts

mb = Marubatsu()
mb.move(1, 1)
print(mb)
boardtxtlist = calc_same_boardtexts(mb, (0, 0))
print("boardtxtlist")
print(boardtxtlist)
tt = {}
print("登録後の置換表の値")
for boardtxt, move in boardtxtlist:
    tt[boardtxt] = (0, 0, move)
    print(tt)

Turn x
...
.O.
...

boardtxtlist
{('....o....', (0, 0)), ('....o....', (2, 2)), ('....o....', (0, 2)), ('....o....', (2, 0))}
登録後の置換表の値
{'....o....': (0, 0, (0, 0))}
{'....o....': (0, 0, (2, 2))}
{'....o....': (0, 0, (0, 2))}
{'....o....': (0, 0, (2, 0))}


In [2]:
def calc_same_boardtexts(mb, move=None):
    data = [ [ 0,  0,  1, 1, -1,  0,  1,  0, -1, 0,  1,  0],
             [ 1, -1,  0, 1,  0, -1] * 2,
             [ 1,  0, -1, 0,  1,  0,  0,  0,  1, 1, -1,  0],
             [ 1, -1,  0, 0,  0,  1] * 2,
             [ 0,  1,  0, 1,  0, -1] * 2,
             [ 1,  0, -1, 1, -1,  0] * 2,
             [ 0,  0,  1, 0,  1,  0] * 2, ]
    if move is None:
        boardtexts = set([mb.board_to_str()])
    else:
        boardtexts = { mb.board_to_str(): move }
    for xa, xb, xc, ya, yb, yc, xa2, xb2, xc2, ya2, yb2, yc2 in data:
        txt = ""
        for x in range(mb.BOARD_SIZE):
            for y in range(mb.BOARD_SIZE):
                txt += mb.board[xa * 2 + xb * x + xc * y][ya * 2 + yb * x + yc * y]
        if move is None:
            boardtexts.add(txt)
        else:
            x, y = move
            x, y = xa2 * 2 + xb2 * x + xc2 * y, ya2 * 2 + yb2 * x + yc2 * y
            boardtexts[txt] = (x, y)
    return boardtexts

In [3]:
mb = Marubatsu()
mb.move(1, 1)
print(mb)
boardtxtlist = calc_same_boardtexts(mb, (0, 0))
print("boardtxtlist")
print(boardtxtlist)
tt = {}
print("登録後の置換表の値")
for boardtxt, move in boardtxtlist.items():
    tt[boardtxt] = (0, 0, move)
    print(tt)

Turn x
...
.O.
...

boardtxtlist
{'....o....': (0, 0)}
登録後の置換表の値
{'....o....': (0, 0, (0, 0))}


In [4]:
from ai import ai_by_mmscore, dprint
from time import perf_counter
from copy import deepcopy

@ai_by_mmscore
def ai_abs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None,
               eval_params={}, use_tt=False, tt=None, tt_for_mo=None):           
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound, _ = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if not use_tt:            
                boardtxt = mborig.board_to_str()
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]        
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                if abscore > score:
                    bestmove = (x, y)
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mb = deepcopy(mborig)
                mb.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                if abscore < score:
                    bestmove = (x, y)
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
#       from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig, bestmove)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt, move in boardtxtlist.items():
                tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [5]:
from time import perf_counter

def ai_ab_iddfs(mb, debug=False, timelimit=10, eval_func=None, eval_params={},
               use_tt=True, share_tt=True, analyze=False, moveordering=True):
    starttime = perf_counter()
    timelimit_pc = starttime + timelimit
    if moveordering:
        use_tt = True
        share_tt = True
        tt_for_mo = {}
    else:
        tt_for_mo = None
    bestmove = mb.calc_legal_moves()[0]
    totalcount = 0
    resultlist = []
    for maxdepth in range(9 - mb.move_count):
        try:
            result = ai_abs_dls(mb, maxdepth=maxdepth, timelimit_pc=timelimit_pc,
                                eval_func=eval_func, eval_params=eval_params,
                                use_tt=use_tt, share_tt=share_tt,
                                analyze=analyze or moveordering, tt_for_mo=tt_for_mo)
        except RuntimeError:
            dprint(debug, "time out")
            break
        if moveordering:
            tt_for_mo = result["tt"]
        totaltime = perf_counter() - starttime
        resultlist.append(result)
        if analyze or moveordering:
            candidate = result["candidate"]
            bestmove = result["bestmove"]
            count = result["count"]
            time = result["time"]
            totalcount += count
            dprint(debug, f"maxdepth: {maxdepth}, time: {time:6.2f}/{totaltime:6.2f} ms, count {count:5}/{totalcount:5}, bestmove: {bestmove}, candidate: {candidate}")
        else:
            bestmove = result
            dprint(debug, f"maxdepth: {maxdepth}, time: {totaltime:6.2f} ms, bestmove: {bestmove}")
    dprint(debug, f"totaltime: {perf_counter() - starttime: 6.2f} ms")
    if analyze:
        return resultlist
    else:
        return bestmove

In [6]:
from ai import ai14s

mb = Marubatsu()
eval_params = {"minimax": True}
ai_ab_iddfs(mb, timelimit=10, debug=True, eval_func=ai14s,
            eval_params=eval_params, analyze=True, moveordering=True);

maxdepth: 0, time:   0.00/  0.00 ms, count     9/    9, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 1, time:   0.01/  0.01 ms, count    33/   42, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 2, time:   0.02/  0.03 ms, count    66/  108, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 3, time:   0.03/  0.05 ms, count   158/  266, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 4, time:   0.02/  0.08 ms, count   269/  535, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 5, time:   0.03/  0.11 ms, count   458/  993, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 6, time:   0.04/  0.14 ms, count   584/ 1577, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 7, time:   0.03/  0.18 ms, count   546/ 2123, bestmove: (1, 0), candidate: [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
maxdepth: 8, time:   0.03/  0.21 ms, count   545/ 2668, bestmove: (2, 2), candidate: [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
totaltime:   0.21 ms


In [1]:
# JupyterLab を再起動後に下記のプログラムを実行する
from ai_new import ai_ab_iddfs, ai14s
from marubatsu import Marubatsu

mb = Marubatsu()
eval_params = {"minimax": True}
ai_ab_iddfs(mb, timelimit=10, debug=True, eval_func=ai14s,
            eval_params=eval_params, analyze=True, moveordering=True);

maxdepth: 0, time:   0.00/  0.00 ms, count     9/    9, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 1, time:   0.01/  0.01 ms, count    33/   42, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 2, time:   0.00/  0.01 ms, count    66/  108, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 3, time:   0.01/  0.02 ms, count   158/  266, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 4, time:   0.02/  0.04 ms, count   269/  535, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 5, time:   0.03/  0.07 ms, count   458/  993, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 6, time:   0.04/  0.10 ms, count   584/ 1577, bestmove: (1, 1), candidate: [(1, 1)]
maxdepth: 7, time:   0.03/  0.14 ms, count   546/ 2123, bestmove: (1, 0), candidate: [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
maxdepth: 8, time:   0.03/  0.17 ms, count   545/ 2668, bestmove: (0, 2), candidate: [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
totaltime:   0.17 ms
